In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np


# Load the CSV files
amplitude_spectrum = pd.read_csv('amplitude_spectrum_RPIn1.csv', header=None)
decibels = pd.read_csv('delta-theta-alpha-beta-RPIn1-decibels.csv', header=None)
spectrum = pd.read_csv('delta_theta_alpha_beta-RPIn1.csv', header=None)
binary_labels = pd.read_csv('r_r_binary.csv')  # Ensure the binary labels match the samples

# Standardize each dataset
scaler = StandardScaler()

amplitude_spectrum_standardized = scaler.fit_transform(amplitude_spectrum)
decibels_standardized = scaler.fit_transform(decibels)
spectrum_standardized = scaler.fit_transform(spectrum)

# Convert binary labels to numpy array
binary_labels = binary_labels['Binary Label'].values  

In [10]:
len(binary_labels)

253

In [11]:
from sklearn.model_selection import KFold

def build_ffnn(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model





In [12]:
from sklearn.metrics import accuracy_score

# Prepare datasets
datasets = {
    'Amplitude Spectrum': amplitude_spectrum_standardized,
    'Decibels': decibels_standardized,
    'Spectrum': spectrum_standardized
}

# Initialize 4-fold cross-validation
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Store results
results = {}

for name, data in datasets.items():
    print(f"\nPerforming 4-Fold Cross-Validation on {name} data...")
    fold_accuracies = []
    fold_losses = []
    
    # Perform cross-validation
    for fold, (train_idx, test_idx) in enumerate(kf.split(data)):
        print(f"\nFold {fold + 1}")
        
        # Split data
        X_train, X_test = data[train_idx], data[test_idx]
        y_train, y_test = binary_labels[train_idx], binary_labels[test_idx]
        
        # Build and train the model
        model = build_ffnn(input_dim=X_train.shape[1])
        history = model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=0)
        
        # Evaluate the model on the test set
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        fold_losses.append(loss)
        fold_accuracies.append(accuracy)
        print(f"Fold {fold + 1} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Store mean performance across folds
    results[name] = {
        'Mean Loss': np.mean(fold_losses),
        'Mean Accuracy': np.mean(fold_accuracies)
    }

# Print final results
print("\nFinal Results:")
for name, metrics in results.items():
    print(f"{name} - Mean Loss: {metrics['Mean Loss']:.4f}, Mean Accuracy: {metrics['Mean Accuracy']:.4f}")



Performing 4-Fold Cross-Validation on Amplitude Spectrum data...

Fold 1


/home/ruilin/Desktop/ECE9063/assignment-1-assignment-group-51/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 1 - Loss: 0.6977, Accuracy: 0.5625

Fold 2
Fold 2 - Loss: 0.6973, Accuracy: 0.5079

Fold 3
Fold 3 - Loss: 0.6841, Accuracy: 0.6190

Fold 4
Fold 4 - Loss: 0.7028, Accuracy: 0.4762

Performing 4-Fold Cross-Validation on Decibels data...

Fold 1
Fold 1 - Loss: 0.6886, Accuracy: 0.6406

Fold 2
Fold 2 - Loss: 0.7055, Accuracy: 0.4603

Fold 3
Fold 3 - Loss: 0.6966, Accuracy: 0.5079

Fold 4
Fold 4 - Loss: 0.7184, Accuracy: 0.4286

Performing 4-Fold Cross-Validation on Spectrum data...

Fold 1
Fold 1 - Loss: 0.6912, Accuracy: 0.4375

Fold 2
Fold 2 - Loss: 0.6900, Accuracy: 0.5079

Fold 3
Fold 3 - Loss: 0.6899, Accuracy: 0.5714

Fold 4
Fold 4 - Loss: 0.6910, Accuracy: 0.4444

Final Results:
Amplitude Spectrum - Mean Loss: 0.6955, Mean Accuracy: 0.5414
Decibels - Mean Loss: 0.7023, Mean Accuracy: 0.5094
Spectrum - Mean Loss: 0.6905, Mean Accuracy: 0.4903
